In [1]:
## IMPORTS

import os
import sys
import time
import math
from datetime import datetime
from io import StringIO

from pathlib import Path

import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from numpy.polynomial.polynomial import Polynomial
import scipy.integrate as integrate
import scipy.optimize as sco
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error
import openpyxl
import networkx as nx
import sklearn.metrics
from IPython.core.display import HTML
import plotly.io as pio
from openpyxl.utils.dataframe import dataframe_to_rows
import openpyxl as opxl
import matplotlib.ticker as mtick
from matplotlib.cm import get_cmap
import time

import copy

from CoolProp.CoolProp import PropsSI

sys.path.append(r'D:\seagu_OneDrive\Documents\GitHub\parallel-flow-distribution-pressure-loss\ansys')
sys.path.append(r'D:\seagu_OneDrive\Documents\GitHub\PVT-thermal-performance-model')
sys.path.append(r'D:\seagu_OneDrive\Documents\GitHub\RD-systems-and-test-benches')
import utils.data_processing as dp

# sys.path.append(r'G:\Mon Drive\GitHub\PVT-thermal-performance-model')
# sys.path.append(r'G:\Mon Drive\GitHub\PVT-PL-model\ansys')

import jou_gen as jg
import ansys_py_bridge as apb
import ansys.fluent.core as pyfluent
import model as ty
import proc as pr
import plot_functions_here as pfun
import heat_transfer as bht
import fluids as fds
import ht
import general as gen

import bridge_module as bm

### Initialisation

In [ ]:
main_fp = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI'
Mesh_fp = os.path.join(main_fp, 'Mesh')
S2S_fp = os.path.join(main_fp, 'S2S')

templates_fp = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\Templates'
geometry_path = templates_fp + '\\' + 'Inputs_TEST_V4.5_GG.xlsx'
hypotheses_path = templates_fp + '\\' + 'Model_hypotheses.xlsx'
testConditions_path = templates_fp + '\\' + 'tCwind.xlsx'
Inputs_PyFluent_path = templates_fp + '\\' + 'Inputs_PyFluent.xlsx'

panelSpecs, hyp, steadyStateConditions_df, Inputs_PyFluent = bm.import_inputs(geometry_path, hypotheses_path, testConditions_path, Inputs_PyFluent_path)
steadyStateConditions_dict = steadyStateConditions_df.to_dict(orient='records')

testConditionsCode = os.path.splitext(os.path.basename(testConditions_path))[0]

tui, solver = bm.init_solver(main_fp, '1610')
nb_it = 60
nb_big_it = 4
T_out_diff_convergence_control = 0.04

In [ ]:
## Change to wind case

jg.change_bc_type(tui, 'cd_fc', 'pressure-outlet')
jg.change_bc_type(tui, 'cd2_horizontal_zd', 'pressure-outlet')
jg.change_bc_type(tui, 'cd2_wind', 'velocity-inlet')

cd2_wind = solver.setup.boundary_conditions.velocity_inlet['cd2_wind'].get_state()
cd2_wind['t']['constant'] = "T_amb"
solver.setup.boundary_conditions.velocity_inlet['cd2_wind'].set_state(cd2_wind)

cd_fc = solver.setup.boundary_conditions.pressure_outlet['cd_fc'].get_state()
cd_fc['avg_press_spec'] = True
solver.setup.boundary_conditions.pressure_outlet['cd_fc'].set_state(cd_fc)

### 1 seule géométrie

In [9]:
### 1 géométrie

# caoMeshName = 'A5_M1__V4.5F2014_s_wHB_r60'
caoMeshName = 'A8_M1__V4.5F5020_s_wHB_r60'
caoMeshCode = caoMeshName.split('__')[0]

hyp['fins_CFD'] = 1.
Inputs = (caoMeshCode, testConditionsCode, panelSpecs, hyp, steadyStateConditions_dict, Inputs_PyFluent)

# bm.init_mesh(tui, Mesh_fp, S2S_fp, caoMeshName)

In [10]:
bm.init_mesh(tui, Mesh_fp, S2S_fp, caoMeshName)

if (hyp['fins_CFD'] == 1.) or (hyp['fins_CFD'] == "1"):
    bm.adapt_for_fins(tui)

In [ ]:
## Boucle cas / Cas par cas AR

bm.simu_bridge_cases(tui, solver, main_fp, Inputs, nb_it, nb_big_it, T_out_diff_convergence_control, method = 'bridge')

In [ ]:
## Boucle cas / Cas par cas UNIFORM

# bm.simu_bridge_cases(tui, solver, main_fp, Inputs, nb_it, nb_big_it, T_out_diff_convergence_control, method = 'uniform')

### AR plusieurs géométries/maillages

In [ ]:
msh_h5_files = glob.glob(Mesh_fp + '/*.msh.h5')
# msh_h5_files = [file for file in msh_h5_files if (("M1" in file) & ("oldM" not in file) & ("plus_TEST" not in file) & ("_r120" not in file))]
msh_h5_files = [file for file in msh_h5_files if (("M1" in file) & ("oldM" not in file) & ("plus_TEST" not in file) & ("_r60" not in file))]

msh_h5_files = [os.path.basename(file_path) for file_path in msh_h5_files]
msh_h5_files = sorted(msh_h5_files, key=lambda x: int(x[1:x.index('_M')]))
msh_h5_files = [file_name.replace(".msh.h5", "") for file_name in msh_h5_files]
# msh_h5_files = [msh_h5_files[2]] + msh_h5_files[3:7] + msh_h5_files[0:2]
# msh_h5_files.append('A10_M1__V4.5F5030_s_wHB_r60')
msh_h5_files

In [ ]:
msh_h5_files = [
                 'A5_M1__V4.5F2014_s_wHB_r60',
                 'A7_M1__V4.5F2020_s_wHB_r60',
                ]
msh_h5_files

In [ ]:
# hyp['fins_CFD'] = 1.
# bm.adapt_for_fins(tui)

In [ ]:
for i, caoMeshName in enumerate(msh_h5_files):

    caoMeshCode = caoMeshName.split('__')[0]

    bm.init_mesh(tui, Mesh_fp, S2S_fp, caoMeshName)

    hyp['fins_CFD'] = 1.

    # if caoMeshCode in ['A3_M1', 'A4_M1']:
    #     hyp['fins_CFD'] = 0.
    # else:
    #     hyp['fins_CFD'] = 1.
    #     bm.adapt_for_fins(tui)

    Inputs = (caoMeshCode, testConditionsCode, copy.deepcopy(panelSpecs), copy.deepcopy(hyp), copy.deepcopy(steadyStateConditions_dict), copy.deepcopy(Inputs_PyFluent))

    bm.simu_bridge_cases(tui, solver, main_fp, Inputs, nb_it, nb_big_it, T_out_diff_convergence_control, method = 'bridge')

In [ ]:
mesh_h5_files = [
                #  'A5_M1__V4.5F2014_s_wHB_r60', # ok
                #  'A7_M1__V4.5F2020_s_wHB_r60', # ok
                #  'A10_M1__V4.5F5030_s_wHB_r60',
                # 'A11_M1__V4.5F5040_s_wHB_r60',
                'A8_M1__V4.5F5020_s_wHB_r60',
                'A4_M1__V4.5NI_s_wHB_r60',
                 ]

### Sans AR

Changer la condition sur l'échangeur pour une température uniforme = T_fluid_1 et choisir method = 'uniform'

In [ ]:
## Boucle cas / Cas par cas AR

bm.simu_bridge_cases(tui, solver, folder_path, hypotheses_path, panelSpecs, Inputs_PyFluent, steadyStateConditions_dict, nb_it, nb_big_it, T_out_diff_convergence_control, method='uniform')

In [ ]:
## Boucle plusieurs géométries/maillages, cas AR

geometries_dict = pd.read_excel(folder_path + '\\' + 'test-geometries.xlsx')
nb_geom = len(geometries_dict)

for geom in range(nb_geom) :

    geom_name = geometries_dict['mesh_name_wo_ext'].iloc[geom]
    folder_path = bm.create_folder_paths(folder_path, geom, case_name = f'cas_{geom_name}')
    bm.init_mesh(tui, geometries_dict, geom)

    bm.simu_bridge_cases(tui, solver, folder_path, hypotheses_path, panelSpecs, Inputs_PyFluent, steadyStateConditions_dict, nb_it, nb_big_it, T_out_diff_convergence_control, method='uniform')

### Juste modèle 1D (df_one)

In [ ]:
## Boucle cas / Cas par cas 1D

folder_path = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests'
geometry_path = folder_path + '\\' + 'Inputs_TEST_V4.5_GG.xlsx'
hypotheses_path = folder_path + '\\' + 'Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'test-Conditions-AR.xlsx'
folder_path_ansys = os.path.abspath(os.path.join(folder_path, '..'))

panelSpecs = pr.import_geometry(geometry_path)
steadyStateConditions_df = pd.read_excel(condi_path,header=2)
stepConditions = steadyStateConditions_df.to_dict(orient='records')
Inputs_PyFluent = pd.read_excel(folder_path + '\\' + 'Inputs_PyFluent.xlsx')

nb_hx = int(apb.get_value('nb_hx', 'named_expression', Inputs_PyFluent))
# nb_case = len(stepConditions)
nb_case = 5
nb_it = 200

# case = 7
# if True :
for case in range(nb_case):
    hyp, big_it, T_amb, T_fluid_in0 = bm.init_hyp(hypotheses_path, stepConditions, case)
    folder_path_case = bm.create_folder_paths(folder_path, case, case_name = 'cas_')
    file_path_result_CFD, file_path_result_PyFluent, file_path_result_df_one, file_path_result_slices_df = bm.create_save_paths(hyp, folder_path_case, big_it)

    df_one, res = ty.simu_one_steady_state_all_he(panelSpecs, stepConditions[case], hyp)

    hyp['method_h_top_g_exchanger'] = 'CFD'
    hyp['method_h_back_abs'] = 'CFD'
    hyp['method_h_back_tube'] = 'CFD'
    hyp['method_h_rad_back_tube'] = 'CFD'

    bm.fill_Pyfluent(Inputs_PyFluent, res, T_fluid_in0, nb_hx)
    bm.fill_temperature(Inputs_PyFluent)

    bm.save_simu_1D(res, nb_hx, file_path_result_df_one, file_path_result_slices_df)

    bm.save_Pyfluents(Inputs_PyFluent, file_path_result_PyFluent)